In [10]:
import cv2
import datetime
import pandas as pd
import numpy as np
import dlib
import os
from matplotlib import pyplot as plt
import skvideo.io
import matplotlib as mpl
import time

In [11]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [15]:
# hyper_parameter
sleep_time = 0.5 # 찍히는 시간
hr = 300 # 가로
vt = 300 # 세로
delay_time = 2 # 각 운동 사이의 딜레이
auto_stop = 1 # 자동종료 설정 (on은 1)

In [17]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
num = int(input('운동 횟수를 입력해주세요'))
playtime = int(input('각 운동당 소요시간을 입력해주세요 (테스트용, 단위 sec)'))


webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

while webcam.isOpened():
    status, frame = webcam.read()
    if status:
        cv2.imshow("test", frame)

    key = cv2.waitKey(1)
    # if key & 0xFF == ord('q'):
    #     break

    if key & 0xFF == ord('s'):
        for i2 in range(num):
            time.sleep(delay_time)
            count = 0
            df_x = pd.DataFrame()
            df_y = pd.DataFrame()
            playtime1 = int(str(round(time.time()))[-4:])
            playsum = playtime1 + playtime
            while True:
                status, frame = webcam.read()
                playtime2 = int(str(round(time.time()))[-4:])
                if playtime2 >= playsum:
                    break
                time.sleep(sleep_time)
                ## resize할 비율 구하기
                img = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                r = hr / img.shape[1]
                dim = (vt, int(img.shape[0] * r))

                resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
                
                rects = detector(resized, 1)
                for i, rect in enumerate(rects):
                    ## 찾은 얼굴의 박스좌표
                    l = rect.left()
                    t = rect.top()
                    b = rect.bottom()
                    r = rect.right()
                    ## facial landmark 찾기
                    shape = predictor(resized, rect)
                    ## facial landmark를 빨간색 점으로 찍어서 표현
                    x_list = []
                    y_list = []
                    for j in range(68):
                        x, y = shape.part(j).x, shape.part(j).y
                        cv2.circle(resized, (x, y), 1, (0, 0, 255), -1)
                        x_list.append(x)
                        y_list.append(y)
                        # cv2.rectangle(resized, (l, t), (r, b), (0, 255, 0), 2)
                        # result = cv2.putText(resized, str(j), (x, y), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.3, (0, 255, 0))
                        ## 얼굴이 있는 부분을 박스쳐주기
                    count += 1
                    df_x['face' + str(count)] = x_list
                    df_y['face' + str(count)] = y_list
                    createFolder('test_' + str(i2))
                    # file = 'test_' + str(i2) + '\\' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f") + '.jpg'
                    # cv2.imwrite(file, result)
                    df_x.to_csv('test_' + str(i2) + '\\test_dfx.csv')
                    df_y.to_csv('test_' + str(i2) + '\\test_dfy.csv')
                    # print(file, ' saved')
            print('df_saved', str(i2))
            del df_x
            del df_y
        print('finish')
        if auto_stop == 1:
            break
    if key & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

df_saved 0
df_saved 1
df_saved 2
finish


In [5]:
df_x

NameError: name 'df_x' is not defined